In [ ]:
!pip install pulp
import pulp
import numpy as np
import pandas as pd
from pulp import*

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.3 MB 4.9 MB/s 


#**MODEL FORMULATION**

In [ ]:
#Data
day = [0,1,2,3,4,5,6] #Day of the Week
cost_block = [1,1,1,1,1,1.25,1.8] #Rate of Cost for different type of day
fixed_cost = [1400,2300,2300,1400,4100]
overtime_cost = [375,750,750,375,1500]

#patient id
patients = np.arange(56).tolist()

#specialty data for each patient
specialty_patients = [2, 3, 2, 3, 0, 1, 1, 1, 0, 4, 3, 2, 2, 2, 0, 1, 0, 0, 0, 3, 2, 0, 0, 1, 4, 2, 2, 3, 2, 1, 4, 0, 2, 0, 1, 2, 2, 1, 3, 0, 2, 0, 2, 2, 4, 4, 0, 2, 1, 0, 2, 0, 0, 0, 1, 3]
specialtyNames_patients = ['ORTHO', 'GEN', 'ORTHO', 'GEN', 'ENT', 'OBGYN', 'OBGYN', 'OBGYN', 'ENT', 'VASCULAR', 'GEN', 'ORTHO', 'ORTHO', 'ORTHO', 'ENT', 'OBGYN', 'ENT', 'ENT', 'ENT', 'GEN', 'ORTHO', 'ENT', 'ENT', 'OBGYN', 'VASCULAR', 'ORTHO', 'ORTHO', 'GEN', 'ORTHO', 'OBGYN', 'VASCULAR', 'ENT', 'ORTHO', 'ENT', 'OBGYN', 'ORTHO', 'ORTHO', 'OBGYN', 'GEN', 'ENT', 'ORTHO', 'ENT', 'ORTHO', 'ORTHO', 'VASCULAR', 'VASCULAR', 'ENT', 'ORTHO', 'OBGYN', 'ENT', 'ORTHO', 'ENT', 'ENT', 'ENT', 'OBGYN', 'GEN']

#Specialty one hot encoding
s0=[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0]
s1=[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
s2=[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
s3=[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
s4=[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

list_s = [list(a) for a in zip(s0, s1, s2, s3, s4)]

#time for each patient
time = [0.46, 1.76, 0.52, 0.44, 0.69, 0.44, 0.34, 0.33, 0.26, 1.13, 0.45, 0.74, 0.36, 1.0, 0.39, 0.91, 0.49, 0.64, 0.51, 1.31, 0.63, 0.38, 0.49, 0.53, 2.62, 0.89, 1.61, 0.7, 0.82, 0.57, 1.05, 0.61, 0.56, 0.34, 0.47, 1.2, 1.13, 0.58, 0.99, 0.3, 0.56, 1.47, 1.07, 1.02, 0.75, 1.88, 0.31, 0.58, 0.39, 0.91, 0.75, 0.2, 1.04, 0.51, 0.56, 0.45]

#overtime for each patient
overtime = [0.27, 0.34, 0.31, 0.6, 0.44, 0.44, 0.54, 0.46, 0.23, 0.39, 0.42, 0.31, 0.46, 0.35, 0.69, 0.43, 0.65, 0.59, 0.5, 0.32, 0.28, 0.38, 0.29, 0.44, 0.49, 0.32, 0.23, 0.5, 0.29, 0.34, 0.31, 0.24, 0.47, 0.45, 0.41, 0.5, 0.37, 0.5, 0.69, 0.68, 0.32, 0.62, 0.39, 0.42, 0.28, 0.28, 0.6, 0.38, 0.34, 0.54, 0.36, 0.62, 0.32, 0.58, 0.27, 0.45]

#number of patients
P = len(patients)

#number of days
D = len(day)

#specialty
specialty = [0, 1, 2, 3, 4]

#number of specialty
S = len(np.unique(specialty))

#Data Dicts
CB = dict(zip(day, cost_block))
f = dict(zip(specialty, fixed_cost))
cs = dict(zip(specialty, overtime_cost))
t = dict(zip(patients, time))
to = dict(zip(patients, overtime))
ps = dict(zip(patients, list_s))


In [ ]:
def get_stochastic(n):
  st=[]
  for i in range(len(time)):
    st1=[]
    for j in range(n):
      st_ovt= np.random.uniform(low=0, high=overtime[i])
      st1.append(time[i] + st_ovt)
    st.append(st1)
  return st

# def get_stochastic(n):
#   overt = []
#   for i in range(len(time)):
#     st1 = []
#     for j in range(n):
#       st_ovt= np.random.uniform(low=0, high=overtime[i])
#       st1.append(st_ovt)
#       overt.extend(st1)
#   st = []
#   for i in range(0, len(time)):
#     st.append(time[i] + overt[i])
#   return st

In [ ]:
# def CreateModel(n):
#   st=get_stochastic(n)
#   scenario=range(n)

#   #Decision Variable
#   X = LpVariable.dicts("Patients's Day", (patients, day, scenario), lowBound=0, upBound=1, cat='Integer')
#   Y = LpVariable.dicts("Specialty's Day", (specialty, day), lowBound=0, upBound=1, cat='Integer')

#   #Dataframe for Results
#   OC = LpVariable.dicts("Overtime Cost", (day, scenario), lowBound=0, cat='Continuous')

#   #Model Definition
#   SurgeryProb = LpProblem("SurgeryProb", LpMinimize)

#   #Objective Function
#   SurgeryProb += lpSum((lpSum(lpSum(CB[d]*([OC[d][o]] + lpSum(f[s]*Y[s][d] for s in specialty)) for d in day))/n) for o in scenario)

#   # Specialty s is open only if at least 1 patient is in specialty s
#   for o in scenario:
#     for d in day:
#       for p in patients:
#         SurgeryProb += X[p][d][o] <= lpSum(ps[p][s]*Y[s][d] for s in specialty)

#   #1 Every patients will be assigned to only one room
#   for o in scenario:
#     for p in patients:
#       SurgeryProb += lpSum(X[p][d][o] for d in day) == 1

#   #3 Every speciality only has at most 1 room for all time periods
#   for d in day:
#     SurgeryProb += lpSum(Y[s][d] for s in specialty) <= 1

#   #4 Overtime cost should be more than 0, if overtime happened
#   for o in scenario:
#     for d in day:
#       SurgeryProb += OC[d][o] >= lpSum(cs[s]*(lpSum(st[p][o]*X[p][d][o]*ps[p][s] for p in patients)-8*Y[s][d]) for s in specialty)

#   #5 Overtime cost should not exceed cost of maximum 2 hours overtime
#   for o in scenario:
#     for d in day:
#       SurgeryProb += OC[d][o] <= 2*(lpSum(cs[s]*Y[s][d] for s in specialty))

#   # 6 Cost of overtime is less than cost of opening new Operating Room (additional)
#   for o in scenario:
#     for s in specialty:
#       for d in day:
#         SurgeryProb += OC[d][o] <= f[s]
  
#   return SurgeryProb

In [ ]:
#Simulasi dengan 1 scenario
def CreateModel(n):
  st=get_stochastic(n)
  scenario=range(n)

  #Decision Variable
  X = LpVariable.dicts("Patients's Day", (patients, day, scenario), lowBound=0, upBound=1, cat='Integer')
  Y = [[0]*len(day)]*len(specialty)
  Y=np.array(Y)
  Y[0][5]=1
  Y[0][6]=1
  Y[1][3]=1
  Y[2][1]=1
  Y[2][2]=1
  Y[3][0]=1
  Y[4][4]=1

  #Y = LpVariable.dicts("Specialty's Day", (specialty, day), lowBound=0, upBound=1, cat='Integer')

  #Dataframe for Results
  OC = LpVariable.dicts("Overtime Cost", (day, scenario), lowBound=0, cat='Continuous')

  #Model Definition
  SurgeryProb = LpProblem("SurgeryProb", LpMinimize)

  #Objective Function
  SurgeryProb += lpSum((lpSum(lpSum(CB[d]*([OC[d][o]] + lpSum(f[s]*Y[s][d] for s in specialty)) for d in day))/n) for o in scenario)

  # Specialty s is open only if at least 1 patient is in specialty s
  for o in scenario:
    for d in day:
      for p in patients:
        SurgeryProb += X[p][d][o] <= lpSum(ps[p][s]*Y[s][d] for s in specialty)

  #1 Every patients will be assigned to only one room
  for o in scenario:
    for p in patients:
      SurgeryProb += lpSum(X[p][d][o] for d in day) == 1

  #3 Every speciality only has at most 1 room for all time periods
  for d in day:
    SurgeryProb += lpSum(Y[s][d] for s in specialty) <= 1

  #4 Overtime cost should be more than 0, if overtime happened
  for o in scenario:
    for d in day:
      SurgeryProb += OC[d][o] >= lpSum(cs[s]*(lpSum(st[p][o]*X[p][d][o]*ps[p][s] for p in patients)-8*Y[s][d]) for s in specialty)

  #5 Overtime cost should not exceed cost of maximum 2 hours overtime
  for o in scenario:
    for d in day:
      SurgeryProb += OC[d][o] <= 2*(lpSum(cs[s]*Y[s][d] for s in specialty))

  # 6 Cost of overtime is less than cost of opening new Operating Room (additional)
  for o in scenario:
    for s in specialty:
      for d in day:
        SurgeryProb += OC[d][o] <= f[s]

  return SurgeryProb

In [ ]:
SurgeryProb=CreateModel(1) 
SurgeryProb.solve()


1

In [ ]:
print("Status:", LpStatus[SurgeryProb.status])
print('Approximated Cost: ', value(SurgeryProb.objective))

Status: Optimal
Approximated Cost:  18078.933455000002


In [ ]:
for v in SurgeryProb.variables():
  if "Specialty's_Day" in v.name: 
    if v.varValue == 1:
      print(v.name, "=", v.varValue)
    if "Overtime_Cost" in v.name: 
      print(v.name, "=", v.varValue)


In [ ]:
def solveCMC(n):
  print('For ', n,' scenarios:')
  SurgeryProb=CreateModel(n) 
  SurgeryProb.solve()
  print("Status:", LpStatus[SurgeryProb.status])
  print('Approximated Cost: ', value(SurgeryProb.objective))
  for v in SurgeryProb.variables():
    if "Specialty's_Day" in v.name: 
      if v.varValue == 1:
        print(v.name, "=", v.varValue)
    if "Overtime_Cost" in v.name: 
      print(v.name, "=", v.varValue)
  return value(SurgeryProb.objective)

In [ ]:
simulation = [5,10,30,50]
for s in simulation:
  print('For ', s, ' scenarios in the simulation:')
  solveCMC(s)

For  5  scenarios in the simulation:
For  5  scenarios:
Status: Optimal
Approximated Cost:  17458.5299794
Overtime_Cost_0_0 = 136.52045
Overtime_Cost_0_1 = 0.0
Overtime_Cost_0_2 = 0.0
Overtime_Cost_0_3 = 0.0
Overtime_Cost_0_4 = 0.0
Overtime_Cost_1_0 = 241.22755
Overtime_Cost_1_1 = 235.10488
Overtime_Cost_1_2 = 0.0
Overtime_Cost_1_3 = 66.030091
Overtime_Cost_1_4 = 60.834096
Overtime_Cost_2_0 = 668.61764
Overtime_Cost_2_1 = 246.45239
Overtime_Cost_2_2 = 0.0
Overtime_Cost_2_3 = 270.93148
Overtime_Cost_2_4 = 251.16752
Overtime_Cost_3_0 = 0.0
Overtime_Cost_3_1 = 0.0
Overtime_Cost_3_2 = 0.0
Overtime_Cost_3_3 = 0.0
Overtime_Cost_3_4 = 0.0
Overtime_Cost_4_0 = 458.37668
Overtime_Cost_4_1 = 549.51616
Overtime_Cost_4_2 = 500.66225
Overtime_Cost_4_3 = 114.00899
Overtime_Cost_4_4 = 143.19972
Overtime_Cost_5_0 = 0.0
Overtime_Cost_5_1 = 0.0
Overtime_Cost_5_2 = 0.0
Overtime_Cost_5_3 = 0.0
Overtime_Cost_5_4 = 0.0
Overtime_Cost_6_0 = 0.0
Overtime_Cost_6_1 = 0.0
Overtime_Cost_6_2 = 0.0
Overtime_Cost_6_3 

In [ ]:
# def simulate(Xvalue, Yvalue, n):
#   SurgeryProb, X, Y = CreateModel(n)
#   for d in day:

#     for p in patients:
#       SurgeryProb += X[p][d] == Xvalue[p][d]

#     for s in specialty:
#       SurgeryProb += Y[s][d] == Yvalue[s][d]

#   SurgeryProb.solve(CPLEX_CMD(msg=0))

#   print("True (Simulated) Cost =", value(SurgeryProb.objective))

# scenarios_optimization = [10, 25, 50]

# scenarios_simulation = [100, 500, 1000]


for sc in scenarios_optimization:
  print('For ', sc, ' scenarios in the optimization:')
  # compute expected cost
  X, Y = solveCMC(sc)
  # compute true cost for different scenarios
  for s in scenarios_simulation:
    print('For ', s, ' scenarios in the simulation:')
    simulate(X, Y, s)

#   print("##################################################################")

In [ ]:
# print('For ', 1,' scenarios:')
# SurgeryProb=CreateModel(1) 
# SurgeryProb.solve()
# print('Approximated Cost: ', value(SurgeryProb.objective))
# print("Status:", LpStatus[SurgeryProb.status])
# for v in SurgeryProb.variables():
#   if "Specialty's_Day" in v.name: 
#     if v.varValue == 1:
#       print(v.name, "=", v.varValue)
#   if "Overtime_Cost" in v.name: 
#     print(v.name, "=", v.varValue)

In [ ]:
# scenarios = [10, 50, 100]

# for s in scenarios:
#   solveCMC(s)

In [ ]:


import numpy as np
import pandas as pd
from pulp import *

"""#**MODEL FORMULATION**"""

#Data
day = [0,1,2,3,4,5,6] #Day of the Week
cost_block = [1,1,1,1,1,1.25,1.8] #Rate of Cost for different type of day
fixed_cost = [1400,2300,2300,1400,4100]
overtime_cost = [375,750,750,375,1500]

#patient id
patients = np.arange(56).tolist()

#specialty data for each patient
specialty_patients = [2, 3, 2, 3, 0, 1, 1, 1, 0, 4, 3, 2, 2, 2, 0, 1, 0, 0, 0, 3, 2, 0, 0, 1, 4, 2, 2, 3, 2, 1, 4, 0, 2, 0, 1, 2, 2, 1, 3, 0, 2, 0, 2, 2, 4, 4, 0, 2, 1, 0, 2, 0, 0, 0, 1, 3]
specialtyNames_patients = ['ORTHO', 'GEN', 'ORTHO', 'GEN', 'ENT', 'OBGYN', 'OBGYN', 'OBGYN', 'ENT', 'VASCULAR', 'GEN', 'ORTHO', 'ORTHO', 'ORTHO', 'ENT', 'OBGYN', 'ENT', 'ENT', 'ENT', 'GEN', 'ORTHO', 'ENT', 'ENT', 'OBGYN', 'VASCULAR', 'ORTHO', 'ORTHO', 'GEN', 'ORTHO', 'OBGYN', 'VASCULAR', 'ENT', 'ORTHO', 'ENT', 'OBGYN', 'ORTHO', 'ORTHO', 'OBGYN', 'GEN', 'ENT', 'ORTHO', 'ENT', 'ORTHO', 'ORTHO', 'VASCULAR', 'VASCULAR', 'ENT', 'ORTHO', 'OBGYN', 'ENT', 'ORTHO', 'ENT', 'ENT', 'ENT', 'OBGYN', 'GEN']

#Specialty one hot encoding
s0=[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0]
s1=[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
s2=[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
s3=[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
s4=[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

list_s = [list(a) for a in zip(s0, s1, s2, s3, s4)]

#time for each patient
time = [0.46, 1.76, 0.52, 0.44, 0.69, 0.44, 0.34, 0.33, 0.26, 1.13, 0.45, 0.74, 0.36, 1.0, 0.39, 0.91, 0.49, 0.64, 0.51, 1.31, 0.63, 0.38, 0.49, 0.53, 2.62, 0.89, 1.61, 0.7, 0.82, 0.57, 1.05, 0.61, 0.56, 0.34, 0.47, 1.2, 1.13, 0.58, 0.99, 0.3, 0.56, 1.47, 1.07, 1.02, 0.75, 1.88, 0.31, 0.58, 0.39, 0.91, 0.75, 0.2, 1.04, 0.51, 0.56, 0.45]

#overtime for each patient
overtime = [0.27, 0.34, 0.31, 0.6, 0.44, 0.44, 0.54, 0.46, 0.23, 0.39, 0.42, 0.31, 0.46, 0.35, 0.69, 0.43, 0.65, 0.59, 0.5, 0.32, 0.28, 0.38, 0.29, 0.44, 0.49, 0.32, 0.23, 0.5, 0.29, 0.34, 0.31, 0.24, 0.47, 0.45, 0.41, 0.5, 0.37, 0.5, 0.69, 0.68, 0.32, 0.62, 0.39, 0.42, 0.28, 0.28, 0.6, 0.38, 0.34, 0.54, 0.36, 0.62, 0.32, 0.58, 0.27, 0.45]

#number of patients
P = len(patients)

#number of days
D = len(day)

#specialty
specialty = [0, 1, 2, 3, 4]

#number of specialty
S = len(np.unique(specialty))

#Data Dicts
CB = dict(zip(day, cost_block))
f = dict(zip(specialty, fixed_cost))
cs = dict(zip(specialty, overtime_cost))
t = dict(zip(patients, time))
to = dict(zip(patients, overtime))
ps = dict(zip(patients, list_s))

def get_stochastic(n):
  overt = []
  for i in range(len(time)):
    st1 = []
    for j in range(n):
      st_ovt= np.random.uniform(low=0, high=overtime[i])
      st1.append(st_ovt)
      overt.extend(st1)
  st = []
  for i in range(0, len(time)):
    st.append(time[i] + overt[i])
  return st

# overt = []
# for i in range(len(time)):
#   st1 = []
#   for j in range(1):
#     st_ovt= np.random.uniform(low=0, high=overtime[i])
#     st1.append(st_ovt)
#     overt.extend(st1)
# st = []
# for i in range(0, len(time)):
#   st.append(time[i] + overt[i])

def CreateModel(n):
  st=get_stochastic(n)
  scenario=range(n)

  #Decision Variable
  X = LpVariable.dicts("Patients's Day", (patients, day), lowBound=0, upBound=1, cat='Integer')
  Y = LpVariable.dicts("Specialty's Day", (specialty, day), lowBound=0, upBound=1, cat='Integer')

  #Dataframe for Results
  OC = LpVariable.dicts("Overtime Cost", (day, scenario), lowBound=0, cat='Continuous')

  #Model Definition
  SurgeryProb = LpProblem("SurgeryProb", LpMinimize)

  #Objective Function
  SurgeryProb += lpSum((lpSum(lpSum(CB[d]*([OC[d][o]] + lpSum(f[s]*Y[s][d] for s in specialty)) for d in day))/n) for o in scenario)

  # Specialty s is open only if at least 1 patient is in specialty s
  for d in day:
    for p in patients:
      SurgeryProb += X[p][d] <= lpSum(ps[p][s]*Y[s][d] for s in specialty)

  #1 Every patients will be assigned to only one room
  for p in patients:
    SurgeryProb += lpSum(X[p][d] for d in day) == 1

  #3 Every speciality only has at most 1 room for all time periods
  for d in day:
    SurgeryProb += lpSum(Y[s][d] for s in specialty) <= 1

  #4 Overtime cost should be more than 0, if overtime happened
  for o in scenario:
    for d in day:
      SurgeryProb += OC[d][o] >= lpSum(cs[s]*(lpSum(st[p]*X[p][d]*ps[p][s] for p in patients)-8*Y[s][d]) for s in specialty)

  #5 Overtime cost should not exceed cost of maximum 2 hours overtime
  for o in scenario:
    for d in day:
      SurgeryProb += OC[d][o] <= 2*(lpSum(cs[s]*Y[s][d] for s in specialty))

  # 6 Cost of overtime is less than cost of opening new Operating Room (additional)
  for o in scenario:
    for s in specialty:
      for d in day:
        SurgeryProb += OC[d][o] <= f[s]
  
  return SurgeryProb, X, Y

def solveCMC(n):
  SurgeryProb, X, Y = CreateModel(n)
  SurgeryProb.solve(CPLEX_CMD(msg=0))
  print('Approximated Cost: ', value(SurgeryProb.objective))

  Xvalue = [0] * len(patients)

  for p in patients:
    Xvalue[p] = [0] * len(day)
    for d in day:
      Xvalue[p][d] = X[p][d].varValue

  Yvalue = [0] * len(specialty)
  for s in specialty:
    Yvalue[s] = [0] * len(day)
    for d in day:
      Yvalue[s][d] = Y[s][d].varValue
  for v in SurgeryProb.variables():
    if "Specialty's_Day" in v.name: 
      if v.varValue == 1:
        print(v.name, "=", v.varValue)
    if "Overtime_Cost" in v.name: 
      print(v.name, "=", v.varValue)
  return Xvalue, Yvalue

def simulate(Xvalue, Yvalue, n):
  SurgeryProb, X, Y = CreateModel(n)
  for d in day:

    for p in patients:
      SurgeryProb += X[p][d] == Xvalue[p][d]

    for s in specialty:
      SurgeryProb += Y[s][d] == Yvalue[s][d]

  SurgeryProb.solve(CPLEX_CMD(msg=0))

  print("True (Simulated) Cost =", value(SurgeryProb.objective))

scenarios_optimization = [10, 25, 50]

scenarios_simulation = [100, 500, 1000]


for sc in scenarios_optimization:
  print('For ', sc, ' scenarios in the optimization:')
  # compute expected cost
  X, Y = solveCMC(sc)
  # compute true cost for different scenarios
  for s in scenarios_simulation:
    print('For ', s, ' scenarios in the simulation:')
    simulate(X, Y, s)

  print("##################################################################")